# Create Op Tutorial

A simple adaptation of using the Jupyter Scala Implementation (https://github.com/alexarchambault/jupyter-scala) of these examples (https://github.com/imagej/imagej-tutorials/tree/master/create-a-new-op)

In [1]:
load.resolver("imagej.public" at "http://maven.imagej.net/content/groups/public")
load.ivy("net.imagej" % "imagej" % "2.0.0-rc-44") // use % since %% adds scala version
load.ivy("de.christophkraemer" % "rhino-script-engine" % "1.1.0")  // since it's not in the classpath with notebooks per default

:: problems summary ::
:::: WARNINGS
	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from sonatype-snapshots, using Sun Oct 25 20:00:40 CET 2015

	Unable to reparse com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT from imagej.public, using Sun Oct 25 20:00:40 CET 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT, using Sun Oct 25 20:00:40 CET 2015 and Sun Oct 25 20:00:40 CET 2015

	Sorting results from com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT, using Sun Oct 25 20:00:40 CET 2015 and Sun Oct 25 20:00:40 CET 2015

	Choosing imagej.public for com.github.alexarchambault.jupyter#jupyter-scala-api_2.10.5;0.2.0-SNAPSHOT

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPSHOT from sonatype-snapshots, using Wed Oct 21 11:44:30 CEST 2015

	Unable to reparse com.github.alexarchambault#ammonite-api_2.10.5;0.3.1-SNAPS

In [2]:
import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import net.imglib2.Cursor
import net.imglib2.img.array.ArrayImg
import net.imglib2.img.array.ArrayImgs
import net.imglib2.img.basictypeaccess.array.DoubleArray
import net.imglib2.`type`.numeric.RealType
import net.imglib2.`type`.numeric.real.DoubleType

import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin

import net.imagej.ops.AbstractOp
import net.imagej.ops.Op
import net.imglib2.Cursor
import net.imglib2.img.array.ArrayImg
import net.imglib2.img.array.ArrayImgs
import net.imglib2.img.basictypeaccess.array.DoubleArray
import net.imglib2.`type`.numeric.RealType
import net.imglib2.`type`.numeric.real.DoubleType
import org.scijava.ItemIO
import org.scijava.plugin.Parameter
import org.scijava.plugin.Plugin

In [3]:
object MyOps {
    @Plugin(`type` = classOf[Op], name = "ramp2")
class Ramp extends AbstractOp {
    
    @Parameter(`type` = ItemIO.OUTPUT)
	private var rampImg: ArrayImg[DoubleType, DoubleArray] = null
    
	override def run(): Unit = {
        rampImg = ArrayImgs.doubles(256, 256);

		val c = rampImg.localizingCursor();
		val pos = new Array[Long](rampImg.numDimensions());
		while (c.hasNext()) {
			c.fwd()
			c.localize(pos)
			c.get().setReal(sum(pos))
		}
	}
    
    private def sum(pos: Array[Long]) = {
		var sum = 0.0f;
		for (p <- pos) sum += p;
		sum
	}
    /** make it accessible from outside **/
    def getOutput() = rampImg
}
}
//import MyOps._


defined object MyOps

In [4]:
import net.imagej.ImageJ
val ij = new ImageJ()

import net.imagej.ImageJ
ij: net.imagej.ImageJ = net.imagej.ImageJ@6b8fe947

In [5]:
import MyOps._
import org.scijava.plugin.PluginInfo
val rampInfo = new PluginInfo(classOf[Ramp],
                              classOf[Op]
                             )
ij.plugin().addPlugin(rampInfo)

import MyOps._
import org.scijava.plugin.PluginInfo
rampInfo: org.scijava.plugin.PluginInfo[net.imagej.ops.Op] = name='ramp2', priority=0.0, enabled=true, pluginType=Op

In [6]:
val rampImage = ij.op().run("ramp2");

// And display the result!
ij.ui().showUI();
ij.ui().show(rampImage);

[ERROR] Cannot create module: cmd2$$user$MyOps$Ramp
org.scijava.module.ModuleException: org.scijava.InstantiableException: java.lang.InstantiationException: cmd2$$user$MyOps$Ramp
	at org.scijava.command.CommandModule.instantiateCommand(CommandModule.java:250)
	at org.scijava.command.CommandModule.<init>(CommandModule.java:96)
	at org.scijava.command.CommandInfo.createModule(CommandInfo.java:330)
	at org.scijava.module.DefaultModuleService.createModule(DefaultModuleService.java:156)
	at net.imagej.ops.DefaultOpMatchingService.createModule(DefaultOpMatchingService.java:341)
	at net.imagej.ops.DefaultOpMatchingService.match(DefaultOpMatchingService.java:288)
	at net.imagej.ops.DefaultOpMatchingService.match(DefaultOpMatchingService.java:155)
	at net.imagej.ops.DefaultOpMatchingService.findMatches(DefaultOpMatchingService.java:142)
	at net.imagej.ops.DefaultOpMatchingService.findModule(DefaultOpMatchingService.java:91)
	at net.imagej.ops.OpEnvironment.module(OpEnvironment.java:226)
	at net

: 

In [7]:
// Hack the Op
val hackedRampImage = {
    val t_op = new Ramp[DoubleType]()
    t_op.run()
    t_op.getOutput()
}
ij.ui().showUI();
ij.ui().show(hackedRampImage);

: 

In [10]:
// Hack the Op
val hackedRefRampImage = {
    val t_op = classOf[MyOps.Ramp].newInstance()
    t_op.run()
    t_op.getOutput()
}
ij.ui().showUI();
ij.ui().show(hackedRefRampImage);

: 

# Constructor Issue
The constructor shows up with an argument via reflection (even though it has no real args). This needs to be fixed.

In [9]:
val myConst = classOf[MyOps.Ramp].getConstructors()
classOf[ImageJ].getConstructors()

myConst: scala.Array[java.lang.reflect.Constructor[_]] = Array(public cmd2$$user$MyOps$Ramp(cmd2$$user$MyOps$))
res6_1: scala.Array[java.lang.reflect.Constructor[_]] = Array(public net.imagej.ImageJ(), public net.imagej.ImageJ(org.scijava.Context))

In [11]:
classOf[MyOps.Ramp].newInstance()

: 

In [12]:
classOf[MyOps.Ramp].newInstance(null)

: 

In [13]:
classOf[MyOps.Ramp].newInstance(Array[Object](null))

: 